<a href="https://colab.research.google.com/github/Duyuyuy/Airbnb_Analysis_Project/blob/main/Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Project - Los Angeles Airbnb data analysis**

Trần Bảo Duy - ITDSIU20061


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!pip install fuzzywuzzy
!pip install pandas_profiling==3.2.0
!pip install folium
!pip install markupsafe==2.0.1

In [ ]:
import os
import re
import random
import numpy as np
import pandas as pd
import folium as fo
import seaborn as sns
import pandas_profiling
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.patches as mpatches
%matplotlib inline

from scipy.stats import linregress
from sklearn.metrics import r2_score
from folium.plugins import BeautifyIcon
from statsmodels.formula.api import logit
from fuzzywuzzy import process
from fuzzywuzzy import fuzz

In [ ]:
os.chdir('/content/gdrive/My Drive/data')

## **I. Data Introduction**

**Raw data collected from [Airbnb](http://insideairbnb.com/get-the-data/):**

In [ ]:
raw_data = pd.read_csv('listings.csv')

### **I.1. Data Size**

The data include 75 columns, listed below:

In [ ]:
raw_data.info()

Along with the length of 45815 rows

In [ ]:
print('This data contains {} rows'.format(len(raw_data)))

### **I.2. Data Columns**

In total of 75 columns, there are 25 columns with worthy insights. Therefore, this project drops out all other 50 columns.

In [ ]:
data = raw_data.drop(columns = ['listing_url', 'scrape_id', 'last_scraped', 'source', 'reviews_per_month', 'number_of_reviews_l30d', 'number_of_reviews_ltm',
                                'calendar_last_scraped', 'availability_90', 'availability_30', 'availability_60', 'has_availability', 'calendar_updated',
                                'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'maximum_nights_avg_ntm',
                                'minimum_nights_avg_ntm', 'host_is_superhost', 'host_about', 'host_location', 'host_name', 'host_since', 'host_url', 'picture_url',
                                'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_thumbnail_url', 'host_picture_url', 'host_verifications',
                                'license', 'calculated_host_listings_count_shared_rooms', 'calculated_host_listings_count_private_rooms',
                                'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count','host_listings_count', 'host_has_profile_pic',
                                'host_neighbourhood', 'first_review', 'last_review', 'bathrooms', 'neighbourhood_group_cleansed', 'neighbourhood', 'bathrooms',
                                'neighborhood_overview', 'name', 'description', 'host_total_listings_count', 'beds'])

The remained data contains 25 columns and 45815 rows.

*Columns:*

In [ ]:
data.info()

*Rows:*

In [ ]:
print('This data contains {} rows'.format(len(data)))

## **II. Data Preprocessing**

### **II.1. Remove null value**

There are 10 columns that contain null values in this data, including:

In [ ]:
for c in data.columns:
  if data[c].isnull().values.any():
    print(c)

This section divides null-contain columns into 3 categories and handles them in 3 different ways.

*Columns with large number of nulls:*

In [ ]:
# Create a view table with columns having large number of nulls:
#replace 0 rating by 1 (min rating=1)

table = data[['host_id', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication',
              'review_scores_location', 'review_scores_value']]
table.replace(to_replace = 0, value = 1, inplace = True)
table = table.groupby('host_id').agg(np.nanmean).reset_index()
table

In [ ]:
# Remove all rows with no data:

table = table[~table.isna().any(axis=1)]
table

In [ ]:
# Merge the table with the data by removing old columns and add columns in the table to the data:

data = data.drop(columns=['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication',
                          'review_scores_location', 'review_scores_value'])
data = data.merge(table, on = 'host_id')
data

*bedrooms column:*

In [ ]:
value = {"bedrooms" : 1} 
data.fillna(value = value, inplace = True)

*Other columns:*

In [ ]:
# Remove nulls in bathroom_text column:

data = data[~data['bathrooms_text'].isna()]

In [ ]:
# Remove nulls in host_identity_verified column:

data = data[~data['host_identity_verified'].isna()]

Data after removing all null values:

In [ ]:
data

### **II.2. Modify column**

Modify *bathrooms_text* column:

In [ ]:

# Converting function:

def convert(n):
  if "half-bath" in str(n):
    return 1
  if re.findall('[\d]*[.]?[\d]*', str(n))[0] != '':
    return int(np.ceil(float(re.findall('[\d]*[.]?[\d]*', str(n))[0])))

In [ ]:
# Add new columns and drop old column:

data['num_of_bath'] = [convert(n) for n in list(data['bathrooms_text'])]
data['bath_type'] = ['share' if 'share' in str(n) else 'private' for n in list(data['bathrooms_text'])]
data = data[~data['num_of_bath'].isnull()]
data.drop('bathrooms_text', axis = 1, inplace = True)

data[['num_of_bath', 'bath_type']]

Modify *property_type* column:

In [ ]:
cat = ['home', 'condo', 'guesthouse', 'rental unit', 'suite', 'villa', 'townhouse' , 'bungalow', 'apartment', 'loft', 'hotel', 'cottage',
       'bed and breakfast', 'resort', 'hostel', 'camper/rv']  
dic = {}
count = 0

for state in cat:  
    matches = process.extract(state, data['property_type'], limit = data.shape[0])
    for potential_match in matches:
        if potential_match[1] >= 80:    


            data.loc[data['property_type'] == potential_match[0], 'property_type'] = state

d = data.loc[data['property_type'].isin(cat) == False , 'property_type'] = 'others'

data['property_type'].unique()

Modify *price* column:

In [ ]:
data['price'] = data['price'].str.replace('\W', '').astype('int64') / 100
data['price']

Create *satisfied* column:

In [ ]:
data['satisfied'] = data.loc[:,['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin',
                                'review_scores_communication', 'review_scores_location', 'review_scores_value']].sum(axis = 1)
data.loc[data['satisfied'] < 32.5, 'satisfied'] = 0
data.loc[data['satisfied'] >= 32.5, 'satisfied'] = 1

Data after modifying columns:

In [ ]:
data

### **II.3. Removing extreme value** 

In [ ]:
data.info()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (15, 4))
sns.boxplot(x = list(data['minimum_nights']), ax = ax[0]); 

Using the multi-condition filter:



```
maximum_nights < 2200 & bedrooms < 10 & price < 4000
```



In [ ]:
filtered_data = data.query("maximum_nights < 2200 & bedrooms < 8 & price < 2500 & num_of_bath< 8 & minimum_nights<360")

Compare the data with the filtered data using boxplot:

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (15, 4))
sns.boxplot(x = list(data['maximum_nights']), ax = ax[0]); 
sns.boxplot(x = list(filtered_data['maximum_nights']), ax = ax[1]); 
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (15, 4))
sns.boxplot(x = list(data['price']), ax = ax[0]); 
sns.boxplot(x = list(filtered_data['price']), ax = ax[1]); 
plt.show() 

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (15, 4))
sns.boxplot(x = list(data['bedrooms']), ax = ax[0]); 
sns.boxplot(x = list(filtered_data['bedrooms']), ax = ax[1]); 
plt.show()

The number of outliers is reduced, therefore the filtered data is better than the old one.

In [ ]:
data = filtered_data

### **II.4. Randomize value** 

Randomize *availability_365* column:

In [ ]:
v = data[data['availability_365'] != 0]
s = data[data['availability_365'] == 0]

filler = v['availability_365'].sample(n = len(s)).tolist()
s['availability_365'] = [a for a in filler]

randomized_data = pd.concat([s, v], ignore_index = True)

Compared randomize data with data using boxplot:

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (15, 4))
sns.histplot(data['availability_365'], ax = ax[0], kde = True); 
sns.histplot(randomized_data['availability_365'], ax = ax[1], kde = True); 
plt.show()

After randomizing, the data has little change but all the zero values are removed

In [ ]:
data = randomized_data

Randomize *number_of_reviews* column:

In [ ]:
data.loc[data['number_of_reviews'] == 0, 'number_of_reviews'] = random.randint(1, 10)

### **II.5. Encoding binomial datadata**

Encode *host_identity_verified* column:

In [ ]:
data['host_identity_verified'] = data['host_identity_verified'].replace({'t' : 1, 'f' : 0})

Encode *bath_type* column:

In [ ]:
data['bath_type'] = data['bath_type'].replace({'private' : 1, 'share' : 0})

Encode *instant_bookable* column:

In [ ]:
data['instant_bookable'] = data['instant_bookable'].replace({'t' : 1, 'f' : 0})